In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import *
from pyspark.sql.functions import log
from pyspark.sql.functions import lag
from pyspark.sql.functions import col

spark=SparkSession.builder.appName('vendor_model').getOrCreate()

dataframe = spark.read.csv("/EDWSTG40/PRC.ORD_H_Purchase_Orders" , inferSchema = True , header = True)
dataframe.show(5)
df = dataframe.select('HPO_PO_Number','HPO_Delivery_Start_Date','HPO_Delivery_End_Date','HPO_BA_Code','HPO_PO_Date')


dataframe1 = spark.read.csv("/EDWSTG40/PRC.ORD_D_Purchase_Orders" , inferSchema = True , header = True)
#dataframe1.show(5)
df1 = dataframe1.select('DPO_Material_Code','DPO_PO_Number')

df2 = df1.join(df, df1.DPO_PO_Number == df.HPO_PO_Number,"left")
df2.show(5)



# dataframe2 = spark.read.csv("/EDWSTG40/MDM.GEM_M_Materials" , inferSchema = True , header = True)
# df10 = dataframe2.select('HPO_PO_Number','HPO_PO_Basic_Value','HPO_PO_Net_Value','HPO_Delivery_Start_Date','HPO_Delivery_End_Date','HPO_BA_Code','HPO_PO_Date')

# dataframe2.show(5)
dataframe3 = spark.read.csv("/EDWSTG40/MDM.BAM_M_Business_Associates" , inferSchema = True , header = True)
df10 = dataframe3.select('MBA_Is_Blacklisted','MBA_BA_Code')
df10.show(5)

dataframe4 = spark.read.csv("/EDWSTG40/WHS.WRH_H_GIN" , inferSchema = True , header = True)
df11 = dataframe4.select('HGIN_PO_Number','HGIN_GIN_Date','HGIN_Supplier_Material_Despatch_Source_City_Code','HGIN_Supplier_Despatch_Source_Town_Pincode','HGIN_Supplier_Material_Despatch_Source_City_Code','HGIN_Warehouse_Code')
df11.show(5)




 
df12 = df2.join(df11, df2.DPO_PO_Number == df11.HGIN_PO_Number,"left")
df12.show(5)

df13 = df12.join(df10,df12.HPO_BA_Code == df10.MBA_BA_Code,"left")
df13.show(5)

dataframe5 = spark.read.csv("/EDWSTG40/MDM.GEM_M_Warehouses" , inferSchema = True , header = True)
df14 = dataframe5.select('MWH_Warehouse_Code','MWH_AB_Code')
df14.show(5)

df20 = df13.join(df14, df13.HGIN_Warehouse_Code == df14.MWH_Warehouse_Code,"left")
df20.show(5)



dataframe6 = spark.read.csv("/EDWSTG40/MDM.GEM_M_Address_Book" , inferSchema = True , header = True)
df15 = dataframe6.select('MAB_AB_Code','MAB_Zip_Code','MAB_City_Code','MAB_State_Code')
df15.show(5)

df21 = df20.join(df15, df20.MWH_AB_Code == df15.MAB_AB_Code,"left")
df21.show(5)

dataframe7 = spark.read.csv("/BFIX1/EDWSTG40/DEN.EDW_MPP_PO_Denormalized" , inferSchema = True , header = True)

df30 = dataframe7.select('PO_No','Actual_Payment_Made_to_Vendor','PO_Amount')
df30.show(5)


df_p = df21.join(df30, df21.HPO_PO_Number == df30.PO_No,"left")
df_p.show(5)


df21 = df_p.drop('HPO_PO_Number')
df21 = df21.drop('HGIN_PO_Number')
df21 = df21.drop('MBA_BA_Code')
df21.show(5)


windowSpec=Window.partitionBy('DPO_Material_Code','HPO_BA_Code').orderBy('DPO_PO_Number')
df4 = df21.withColumn("row_number",row_number().over(windowSpec))
# df4.show(5)

df4.show(1)

df6 = df4.select("*").groupby('DPO_Material_Code','HPO_BA_Code').agg(max('row_number').alias("total_time"))
print("df6")
df6.show(5)

# # df6.createOrReplaceTempView("df6")

# # df_6=spark.sql("Select DPO_Material_Code as Material_Code , HPO_BA_Code as BA_Code , total_time from df6")
# df_6 = df6.withColumn("Material_Code",DPO_Material_Code)
# df_61 = df_6.withColumn("BA_Code",HPO_BA_Code)
# # print("df_6
# df_61.show(5)

df61 = df6.withColumnRenamed("DPO_Material_Code","Material_Code")
df62 = df61.withColumnRenamed("HPO_BA_Code","BA_Code")
df62.show(5)

df_collect=df4.join(df62,df4.HPO_BA_Code==df62.BA_Code,"left")

df_collect.show(30)
# df_collect = df_collect.drop('DPO_Material_Code')
# df_collect = df_collect.drop('HPO_BA_Code')
# df_collect.show(15)

df_collect = df_collect.filter(df_collect.MBA_Is_Blacklisted != 'Y')


df_collect = df_collect.withColumn("Actual_Delivery", when((col("HGIN_GIN_Date") >= col("HPO_Delivery_Start_Date")) & (col("HGIN_GIN_Date") <= col("HPO_Delivery_End_Date")), "yes").otherwise("no"))
df_collect.show(5)

df_collect = df_collect.withColumn("Same_PIN", when(col("HGIN_Supplier_Despatch_Source_Town_Pincode") == col("MAB_Zip_Code"), "yes").otherwise("no"))
df_collect = df_collect.withColumn("Same_City", when(col("HGIN_Supplier_Despatch_Source_City_Pincode") == col("MAB_City_Code"), "yes").otherwise("no"))
df_collect = df_collect.withColumn("Same_State", when(col("HGIN_Supplier_Despatch_Source_State_Pincode") == col("MAB_State_Code"), "yes").otherwise("no"))
df_collect.show(5)


df_collect = df_collect.withColumn("Price_Diff", col("PO_Amount") - col("Actual_Payment_Made_to_Vendor"))
df_collect.show(5)

df_final = df_collect.select('Actual_Delivery','Same_PIN','Same_City','Same_State','total_time_vendor_is_choosen','price_diff','BA_code')
#df_final.show(5)


from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator



label_indexer1 = StringIndexer(inputCol="Actual_Delivery", outputCol="Delay")
df_final = label_indexer1.fit(df_final).transform(df_final)

label_indexer2 = StringIndexer(inputCol="Same_PIN", outputCol="Same_PINs")
df_final = label_indexer2.fit(df_final).transform(df_final)

label_indexer3 = StringIndexer(inputCol="Same_City", outputCol="Same_Citys")
df_final = label_indexer3.fit(df_final).transform(df_final)

label_indexer4 = StringIndexer(inputCol="Same_State", outputCol="Same_States")
df_final  = label_indexer4.fit(df_final).transform(df_final)

label_indexer5 = StringIndexer(inputCol="BA_code", outputCol="vendor")
df_final  = label_indexer5.fit(df_final).transform(df_final)

df_final.show(5)

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

df_reg = df_final.drop('BA_Code')
assembler = VectorAssembler(inputCols = df_reg.columns, outputCol = 'features')
output=assembler.transform(df_final)

df_ml = output.select(['features' , 'BA_Code'])
df_ml.show(5)

train_data, test_data = output.randomSplit([0.8, 0.2])


rf = RandomForestClassifier(labelCol="BA_Code", featuresCol="features")

model = rf.fit(train_data)

predictions = model.transform(test_data)
predictions.show(5)

-- from pyspark.ml.evaluation import MulticlassClassificationEvaluator

-- evaluator = MulticlassClassificationEvaluator(labelCol="BA_Code", predictionCol="prediction")
-- accuracy = evaluator.evaluate(predictions)
-- print("Accuracy = %s" % (accuracy))
-- print("Test Error = %s" % (1.0 - accuracy))

SyntaxError: unexpected character after line continuation character (2407452009.py, line 115)